### This is the notebook to run the Open LLM's as an Web Service

In [0]:
%run "./util/notebook-config"

**This notebook is not required if you want to run the OpenAI model**

In [0]:
if config['model_id'] == "openai":
  raise "Notebook note required , Use this notebook to run on when using open LLM. change the config"

In [0]:
%pip install torch==2.0.1

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (173.2 MB)
  Using cached nvidia_nvtx_cu11-11.7.91-py3-none-manylinux1_x86_64.whl (98 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_curand_cu11-10.2.10.91-py3-none-manylinux1_x86_64.whl (54.6 MB)
  Using cached sympy-1.12-py3-none-an

In [0]:
# ! rm -rf /dbfs/$user/tgi/*

In [0]:
%sh
# install rust
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs > tmp.sh
sh tmp.sh -y
source "$HOME/.cargo/env"

# install protoc
PROTOC_ZIP=protoc-21.12-linux-x86_64.zip
curl -OL https://github.com/protocolbuffers/protobuf/releases/download/v21.12/$PROTOC_ZIP
sudo unzip -o $PROTOC_ZIP -d /usr/local bin/protoc
sudo unzip -o $PROTOC_ZIP -d /usr/local 'include/*'
rm -f $PROTOC_ZIP

# install text-generation-inference
rm -rf  /local_disk0/tmp/text-generation-inference
cd /local_disk0/tmp && git clone https://github.com/huggingface/text-generation-inference.git  
cd /local_disk0/tmp/text-generation-inference && make install

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu'



  stable-x86_64-unknown-linux-gnu unchanged - rustc 1.72.0 (5680fa18f 2023-08-23)


Rust is installed now. Great!

To get started you may need to restart your current shell.
This would reload your PATH environment variable to include
Cargo's bin directory ($HOME/.cargo/bin).

To configure your current shell, run:
source "$HOME/.cargo/env"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1548k  100 1548k    0     0  7315k      0 --:--:-- --:--:-- --:--:-- 7315k


Archive:  protoc-21.12-linux-x86_64.zip
  inflating: /usr/local/bin/protoc   
Archive:  protoc-21.12-linux-x86_64.zip
  inflating: /usr/local/include/google/protobuf/any.proto  
  inflating: /usr/local/include/google/protobuf/api.proto  
  inflating: /usr/local/include/google/protobuf/compiler/plugin.proto  
  inflating: /usr/local/include/google/protobuf/descriptor.proto  
  inflating: /usr/local/include/google/protobuf/duration.proto  
  inflating: /usr/local/include/google/protobuf/empty.proto  
  inflating: /usr/local/include/google/protobuf/field_mask.proto  
  inflating: /usr/local/include/google/protobuf/source_context.proto  
  inflating: /usr/local/include/google/protobuf/struct.proto  
  inflating: /usr/local/include/google/protobuf/timestamp.proto  
  inflating: /usr/local/include/google/protobuf/type.proto  
  inflating: /usr/local/include/google/protobuf/wrappers.proto  


Cloning into 'text-generation-inference'...


...

make[1]: Leaving directory '/local_disk0/tmp/text-generation-inference/server'
cd router && cargo install --path .


  Installing text-generation-router v1.0.3 (/local_disk0/tmp/text-generation-inference/router)
    Updating crates.io index
   Compiling libc v0.2.147
   Compiling proc-macro2 v1.0.66
   Compiling unicode-ident v1.0.11
   Compiling cfg-if v1.0.0
   Compiling autocfg v1.1.0
   Compiling version_check v0.9.4
   Compiling once_cell v1.18.0
   Compiling pin-project-lite v0.2.13
   Compiling memchr v2.6.2
   Compiling scopeguard v1.2.0
   Compiling futures-core v0.3.28
   Compiling pkg-config v0.3.27
   Compiling bytes v1.4.0
   Compiling futures-sink v0.3.28
   Compiling smallvec v1.11.0
   Compiling futures-channel v0.3.28
   Compiling futures-io v0.3.28
   Compiling futures-task v0.3.28
   Compiling typenum v1.16.0
   Compiling parking_lot_core v0.9.8
   Compiling futures-util v0.3.28
   Compiling pin-utils v0.1.0
   Compiling serde v1.0.188
   Compiling log v0.4.20
   Compiling itoa v1.0.9
   Compiling fnv v1.0.7
   Compiling hashbrown v0.12.3
   Compiling syn v1.0.109
   Compiling ppv-

cd launcher && cargo install --path .


  Installing text-generation-launcher v1.0.3 (/local_disk0/tmp/text-generation-inference/launcher)
    Updating crates.io index
   Compiling once_cell v1.18.0
   Compiling semver v1.0.18
   Compiling memoffset v0.7.1
   Compiling regex-syntax v0.7.5
   Compiling serde_json v1.0.105
   Compiling syn v2.0.29
   Compiling vergen v8.2.4
   Compiling sysinfo v0.29.9
   Compiling tracing-core v0.1.31
   Compiling thread_local v1.1.7
   Compiling nix v0.26.4
   Compiling tracing-log v0.1.3
   Compiling rustc_version v0.4.0
   Compiling text-generation-launcher v1.0.3 (/local_disk0/tmp/text-generation-inference/launcher)
   Compiling regex-automata v0.3.7
   Compiling regex v1.9.4
   Compiling ctrlc v3.4.0
   Compiling serde_derive v1.0.188
   Compiling tracing-attributes v0.1.26
   Compiling clap_derive v4.4.0
   Compiling tracing v0.1.37
   Compiling clap v4.4.1
   Compiling serde v1.0.188
   Compiling tracing-serde v0.1.3
   Compiling tracing-subscriber v0.3.17
    Finished release [optimiz

if [ "$BUILD_EXTENSIONS" = "True" ]; then cd server/custom_kernels && python setup.py install; else echo "Custom kernels are disabled, you need to set the BUILD_EXTENSIONS environment variable to 'True' in order to build them. (Please read the docs, kernels might not work on all hardware)"; fi
Custom kernels are disabled, you need to set the BUILD_EXTENSIONS environment variable to 'True' in order to build them. (Please read the docs, kernels might not work on all hardware)


In [0]:
%sh 
FILE=/dbfs/$user/tgi/flash_attn-1.0.8-cp310-cp310-linux_x86_64.whl 
if test -f "$FILE"; then
    echo "$FILE exists."
else
    export flash_att_commit='3a9bfd076f98746c73362328958dbc68d145fbec'
    mkdir /dbfs/$user/tgi/  -p
    rm -rf  /local_disk0/tmp/flash-attention
    cd /local_disk0/tmp && git clone https://github.com/HazyResearch/flash-attention.git 

    cd flash-attention && git fetch && git checkout ${flash_att_commit}
    python setup.py build
    python setup.py bdist_wheel
    cp  dist/flash_attn-1.0.8-cp310-cp310-linux_x86_64.whl /dbfs/$user/tgi/flash_attn-1.0.8-cp310-cp310-linux_x86_64.whl
    cd csrc/rotary && python setup.py build 
    python setup.py bdist_wheel
    cp  dist/rotary_emb-0.1-cp310-cp310-linux_x86_64.whl /dbfs/$user/tgi/rotary_emb-0.1-cp310-cp310-linux_x86_64.whl
    cd ..
    cd layer_norm && python setup.py build 
    python setup.py bdist_wheel
    cp  dist/dropout_layer_norm-0.1-cp310-cp310-linux_x86_64.whl /dbfs/$user/tgi/dropout_layer_norm-0.1-cp310-cp310-linux_x86_64.whl
fi

Cloning into 'flash-attention'...
Note: switching to '3a9bfd076f98746c73362328958dbc68d145fbec'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 3a9bfd0 [FT] rotary_cos/sin should have shape (dim) instead of (seqlen, dim)
Submodule 'csrc/flash_attn/cutlass' (https://github.com/NVIDIA/cutlass.git) registered for path 'csrc/flash_attn/cutlass'
Cloning into '/local_disk0/tmp/flash-attention/csrc/flash_attn/cutlass'...


Submodule path 'csrc/flash_attn/cutlass': checked out '319a389f42b776fae5701afcb943fc03be5b5c25'


torch.__version__  = 2.0.1+cu117


running build
running build_py
creating build
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/flash_blocksparse_attn_interface.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/flash_attn_triton_og.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/flash_attn_triton.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/flash_attention.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/flash_attn_interface.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/fused_softmax.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/__init__.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/bert_padding.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying 

Emitting ninja build file /local_disk0/tmp/flash-attention/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/9] c++ -MMD -MF /local_disk0/tmp/flash-attention/build/temp.linux-x86_64-cpython-310/csrc/flash_attn/fmha_api.o.d -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/local_disk0/tmp/flash-attention/csrc/flash_attn -I/local_disk0/tmp/flash-attention/csrc/flash_attn/src -I/local_disk0/tmp/flash-attention/csrc/flash_attn/cutlass/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/TH -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/THC -I/usr/local/cuda/include -I/local

Emitting ninja build file /local_disk0/tmp/flash-attention/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


...

[1/57] c++ -MMD -MF /local_disk0/tmp/flash-attention/csrc/layer_norm/build/temp.linux-x86_64-cpython-310/ln_api.o.d -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/local_disk0/tmp/flash-attention/csrc/layer_norm -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/TH -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/THC -I/usr/local/cuda/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/include -I/usr/include/python3.10 -c -c /local_di

Emitting ninja build file /local_disk0/tmp/flash-attention/csrc/layer_norm/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
x86_64-linux-gnu-g++ -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 /local_disk0/tmp/flash-attention/csrc/layer_norm/build/temp.linux-x86_64-cpython-310/ln_api.o /local_disk0/tmp/flash-attention/csrc/layer_norm/build/temp.linux-x86_64-cpython-310/ln_bwd_1024.o /local_disk0/tmp/flash-attention/csrc/layer_norm/build/temp.linux-x86_64-cpython-310/ln_bwd_1280.o /local_disk0/tmp/flash-attention/csrc/layer_norm/build/temp.linux-x86_64-cpython-310/ln_bwd_1536.o /local_disk0/tmp/flash-attention/csrc/layer_norm/build/temp.linux-x86_64-cpython-310/ln_bwd_2048.o /local_disk0/tmp/flash-attention/csrc/layer_norm/build/temp.linux-x86_64-cpython-310/ln_bwd_256.o /local_disk0/tmp/flash-attention/csrc/layer_norm/build/temp.linux-x86_64-cpython-310/ln_bwd_2560.o /local_disk0/tmp/flash-attention/csrc/layer_norm/build/temp.linux-x86_64-cpython-310/ln_bwd_3072.o /local_disk0/tmp/flash-attention/csrc/layer_norm/build/temp.linux-x86_64-cpython-310/ln_bw

/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


installing to build/bdist.linux-x86_64/wheel
running install
running install_lib
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/wheel
copying build/lib.linux-x86_64-cpython-310/dropout_layer_norm.cpython-310-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/wheel
running install_egg_info
running egg_info
creating dropout_layer_norm.egg-info
writing dropout_layer_norm.egg-info/PKG-INFO
writing dependency_links to dropout_layer_norm.egg-info/dependency_links.txt
writing top-level names to dropout_layer_norm.egg-info/top_level.txt
writing manifest file 'dropout_layer_norm.egg-info/SOURCES.txt'
reading manifest file 'dropout_layer_norm.egg-info/SOURCES.txt'
writing manifest file 'dropout_layer_norm.egg-info/SOURCES.txt'
Copying dropout_layer_norm.egg-info to build/bdist.linux-x86_64/wheel/dropout_layer_norm-0.1-py3.10.egg-info
running install_scripts
creating build/bdist.linux-x86_64/wheel/dropout_layer_norm-0.1.dist-info/WHEEL
creating 'dist/dropout_layer_norm-0.1-cp310

In [0]:
%sh 
FILE=/dbfs/$user/tgi/flash_attn-2.0.0.post1-cp310-cp310-linux_x86_64.whl
if test -f "$FILE"; then
    echo "$FILE exists."
else
    export flash_att_v2_commit='4f285b354796fb17df8636485b9a04df3ebbb7dc'

    rm -rf  /local_disk0/tmp/flash-attention-v2
    cd /local_disk0/tmp && git clone https://github.com/HazyResearch/flash-attention.git flash-attention-v2

    cd flash-attention-v2 && git fetch && git checkout ${flash_att_v2_commit}
    python setup.py build
    python setup.py bdist_wheel
    cp  dist/flash_attn-2.0.0.post1-cp310-cp310-linux_x86_64.whl /dbfs/$user/tgi/flash_attn-2.0.0.post1-cp310-cp310-linux_x86_64.whl
fi

Cloning into 'flash-attention-v2'...
Note: switching to '4f285b354796fb17df8636485b9a04df3ebbb7dc'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 4f285b3 FlashAttention-2 release
Submodule 'csrc/cutlass' (https://github.com/NVIDIA/cutlass.git) registered for path 'csrc/cutlass'
Cloning into '/local_disk0/tmp/flash-attention-v2/csrc/cutlass'...


Submodule path 'csrc/cutlass': checked out 'c4f6b8c6bc94ff69048492fb34df0dfaf1983933'


torch.__version__  = 2.0.1+cu117


running build
running build_py
creating build
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/flash_blocksparse_attn_interface.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/flash_attn_triton_og.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/flash_attn_triton.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/flash_attn_interface.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/fused_softmax.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/__init__.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/bert_padding.py -> build/lib.linux-x86_64-cpython-310/flash_attn
copying flash_attn/flash_blocksparse_attention.py -> build/lib.linux-x86_64-cpython-310/flash_attn
creatin

Emitting ninja build file /local_disk0/tmp/flash-attention-v2/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/33] c++ -MMD -MF /local_disk0/tmp/flash-attention-v2/build/temp.linux-x86_64-cpython-310/csrc/flash_attn/flash_api.o.d -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/local_disk0/tmp/flash-attention-v2/csrc/flash_attn -I/local_disk0/tmp/flash-attention-v2/csrc/flash_attn/src -I/local_disk0/tmp/flash-attention-v2/csrc/cutlass/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/TH -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/THC -I/usr/local/cuda/include -I/lo

Emitting ninja build file /local_disk0/tmp/flash-attention-v2/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
x86_64-linux-gnu-g++ -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 /local_disk0/tmp/flash-attention-v2/build/temp.linux-x86_64-cpython-310/csrc/flash_attn/flash_api.o /local_disk0/tmp/flash-attention-v2/build/temp.linux-x86_64-cpython-310/csrc/flash_attn/src/flash_bwd_hdim128_bf16_sm80.o /local_disk0/tmp/flash-attention-v2/build/temp.linux-x86_64-cpython-310/csrc/flash_attn/src/flash_bwd_hdim128_fp16_sm80.o /local_disk0/tmp/flash-attention-v2/build/temp.linux-x86_64-cpython-310/csrc/flash_attn/src/flash_bwd_hdim160_bf16_sm80.o /local_disk0/tmp/flash-attention-v2/build/temp.linux-x86_64-cpython-310/csrc/flash_attn/src/flash_bwd_hdim160_fp16_sm80.o /local_disk0/tmp/flash-attention-v2/build/temp.linux-x86_64-cpython-310/csrc/flash_attn/src/flash_bwd_hdim192_bf16_sm80.o /local_disk0/tmp/flash-attention-v2/build/temp.linux-x86_64-cpython-310/csrc/flash_attn/src/flash_bwd_hdim192_fp16_sm80.o /local_disk0/tmp/flash-attention-v2/build/te

/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


installing to build/bdist.linux-x86_64/wheel
running install
running install_lib
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/wheel
copying build/lib.linux-x86_64-cpython-310/flash_attn_2_cuda.cpython-310-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/wheel
creating build/bdist.linux-x86_64/wheel/flash_attn
creating build/bdist.linux-x86_64/wheel/flash_attn/ops
copying build/lib.linux-x86_64-cpython-310/flash_attn/ops/fused_dense.py -> build/bdist.linux-x86_64/wheel/flash_attn/ops
copying build/lib.linux-x86_64-cpython-310/flash_attn/ops/rms_norm.py -> build/bdist.linux-x86_64/wheel/flash_attn/ops
copying build/lib.linux-x86_64-cpython-310/flash_attn/ops/__init__.py -> build/bdist.linux-x86_64/wheel/flash_attn/ops
copying build/lib.linux-x86_64-cpython-310/flash_attn/ops/layer_norm.py -> build/bdist.linux-x86_64/wheel/flash_attn/ops
copying build/lib.linux-x86_64-cpython-310/flash_attn/ops/activations.py -> build/bdist.linux-x86_64/wheel/flash_attn/ops
copying b

adding license file 'LICENSE'
adding license file 'AUTHORS'
writing manifest file 'flash_attn.egg-info/SOURCES.txt'
Copying flash_attn.egg-info to build/bdist.linux-x86_64/wheel/flash_attn-2.0.0.post1-py3.10.egg-info
running install_scripts
adding license file "LICENSE" (matched pattern "LICEN[CS]E*")
adding license file "AUTHORS" (matched pattern "AUTHORS*")
creating build/bdist.linux-x86_64/wheel/flash_attn-2.0.0.post1.dist-info/WHEEL
creating 'dist/flash_attn-2.0.0.post1-cp310-cp310-linux_x86_64.whl' and adding 'build/bdist.linux-x86_64/wheel' to it
adding 'flash_attn_2_cuda.cpython-310-x86_64-linux-gnu.so'
adding 'flash_attn/__init__.py'
adding 'flash_attn/bert_padding.py'
adding 'flash_attn/flash_attn_interface.py'
adding 'flash_attn/flash_attn_triton.py'
adding 'flash_attn/flash_attn_triton_og.py'
adding 'flash_attn/flash_blocksparse_attention.py'
adding 'flash_attn/flash_blocksparse_attn_interface.py'
adding 'flash_attn/fused_softmax.py'
adding 'flash_attn/layers/__init__.py'
ad

In [0]:
%sh 
FILE=/dbfs/$user/tgi/vllm-0.0.0-cp310-cp310-linux_x86_64.whl
if test -f "$FILE"; then
    echo "$FILE exists."
else
    export vllm_commit='d284b831c17f42a8ea63369a06138325f73c4cf9'

    rm -rf  /local_disk0/tmp/vllm
    cd /local_disk0/tmp && git clone https://github.com/OlivierDehaene/vllm.git

    cd vllm && git fetch && git checkout ${vllm_commit}
    python setup.py build
    python setup.py bdist_wheel
    cp dist/vllm-0.0.0-cp310-cp310-linux_x86_64.whl /dbfs/$user/tgi/vllm-0.0.0-cp310-cp310-linux_x86_64.whl
fi

Cloning into 'vllm'...
Note: switching to 'd284b831c17f42a8ea63369a06138325f73c4cf9'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at d284b83 add kv_head_mapping for grouped attention


running build
running build_ext
building 'vllm_cache_ops' extension
creating /local_disk0/tmp/vllm/build
creating /local_disk0/tmp/vllm/build/temp.linux-x86_64-cpython-310
creating /local_disk0/tmp/vllm/build/temp.linux-x86_64-cpython-310/csrc


Emitting ninja build file /local_disk0/tmp/vllm/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] c++ -MMD -MF /local_disk0/tmp/vllm/build/temp.linux-x86_64-cpython-310/csrc/cache.o.d -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/TH -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/include/THC -I/usr/local/cuda/include -I/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/include -I/usr/include/python3.10 -c -c /local_disk0/tmp/vllm/csrc/cache.cpp -o /local_disk0/tmp/vllm/build/temp.linux-x86_6

Emitting ninja build file /local_disk0/tmp/vllm/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


...

ninja: no work to do.
x86_64-linux-gnu-g++ -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 /local_disk0/tmp/vllm/build/temp.linux-x86_64-cpython-310/csrc/cache.o /local_disk0/tmp/vllm/build/temp.linux-x86_64-cpython-310/csrc/cache_kernels.o -L/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/lib -L/usr/local/cuda/lib64 -L/usr/lib/x86_64-linux-gnu -lc10 -ltorch -ltorch_cpu -ltorch_python -lcudart -lc10_cuda -ltorch_cuda -o build/lib.linux-x86_64-cpython-310/vllm_cache_ops.cpython-310-x86_64-linux-gnu.so
building 'vllm_attention_ops' extension


Emitting ninja build file /local_disk0/tmp/vllm/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
x86_64-linux-gnu-g++ -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 /local_disk0/tmp/vllm/build/temp.linux-x86_64-cpython-310/csrc/attention.o /local_disk0/tmp/vllm/build/temp.linux-x86_64-cpython-310/csrc/attention/attention_kernels.o -L/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/torch/lib -L/usr/local/cuda/lib64 -L/usr/lib/x86_64-linux-gnu -lc10 -ltorch -ltorch_cpu -ltorch_python -lcudart -lc10_cuda -ltorch_cuda -o build/lib.linux-x86_64-cpython-310/vllm_attention_ops.cpython-310-x86_64-linux-gnu.so


/local_disk0/.ephemeral_nfs/envs/pythonEnv-0cc3237d-083d-4b83-a77a-666d859a692b/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


installing to build/bdist.linux-x86_64/wheel
running install
running install_lib
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/wheel
copying build/lib.linux-x86_64-cpython-310/vllm_cache_ops.cpython-310-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/wheel
copying build/lib.linux-x86_64-cpython-310/vllm_attention_ops.cpython-310-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/wheel
running install_egg_info
running egg_info
creating vllm.egg-info
writing vllm.egg-info/PKG-INFO
writing dependency_links to vllm.egg-info/dependency_links.txt
writing top-level names to vllm.egg-info/top_level.txt
writing manifest file 'vllm.egg-info/SOURCES.txt'
reading manifest file 'vllm.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'vllm.egg-info/SOURCES.txt'
Copying vllm.egg-info to build/bdist.linux-x86_64/wheel/vllm-0.0.0-py3.10.egg-info
running install_scripts
adding license file "LICENSE" (matched pattern "LICE

In [0]:
%pip install /dbfs/$user/tgi/flash_attn-2* /dbfs/$user/tgi/dropout_laye* /dbfs/$user/tgi/rotary_emb*  /dbfs/$user/tgi/vllm*  urllib3==1.25.4 protobuf==3.20.*

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Processing /dbfs/tgi/flash_attn-2.0.0.post1-cp310-cp310-linux_x86_64.whl
Processing /dbfs/tgi/dropout_layer_norm-0.1-cp310-cp310-linux_x86_64.whl
Processing /dbfs/tgi/rotary_emb-0.1-cp310-cp310-linux_x86_64.whl
Processing /dbfs/tgi/vllm-0.0.0-cp310-cp310-linux_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.5/125.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.1 MB/s eta 0:00:00
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
    

In [0]:
#  dbutils.library.restartPython() 

In [0]:
import os 
nodeid = spark.conf.get('spark.databricks.driverNodeTypeId')
if "A100" in nodeid:
  os.environ['sharded'] = 'false'
  os.environ['CUDA_VISIBLE_DEVICES'] = "0"
else:
  os.environ['sharded'] = 'true'
  os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3"

if "Llama-2" in config['model_id']: 
  os.environ['HUGGING_FACE_HUB_TOKEN'] = config['HUGGING_FACE_HUB_TOKEN']
os.environ['HUGGINGFACE_HUB_CACHE'] ='/local_disk0/tmp/'
os.environ['CUDA_MEMORY_FRACTION'] = "0.95"

# get model variables
os.environ['model_id'] = config['model_id']
if "load_in_8bit" in config['model_kwargs']:
  os.environ['quantize'] = "bitsandbytes"
if config['model_id'] != 'meta-llama/Llama-2-70b-chat-hf':
  os.environ['CUDA_MEMORY_FRACTION'] = ".9"


In [0]:
from dbruntime.databricks_repl_context import get_context
ctx = get_context()

port = "8880"
driver_proxy_api = f"https://{ctx.browserHostName}/driver-proxy-api/o/0/{ctx.clusterId}/{port}"

print(f"""
driver_proxy_api = '{driver_proxy_api}'
cluster_id = '{ctx.clusterId}'
port = {port}
""")


driver_proxy_api = 'https://adb-984752964297111.11.azuredatabricks.net/driver-proxy-api/o/0/0728-153932-k08t5wf6/8880'
cluster_id = '0728-153932-k08t5wf6'
port = 8880



In [0]:
! echo $CUDA_MEMORY_FRACTION

.9


In [0]:
%sh
source "$HOME/.cargo/env"

if [ -z ${quantize} ]; 
    then echo "quantize" && text-generation-launcher --model-id $model_id --port 8880 --trust-remote-code --sharded $sharded --max-input-length 2048 --max-total-tokens 4096 ;
else text-generation-launcher --model-id $model_id --port 8880 --trust-remote-code --sharded $sharded --max-input-length 2048 --max-total-tokens 4096 --quantize bitsandbytes  ;
fi


quantize
2023-08-31T09:26:42.273434Z  INFO text_generation_launcher: Args { model_id: "mosaicml/mpt-30b-chat", revision: None, validation_workers: 2, sharded: Some(false), num_shard: None, quantize: None, dtype: None, trust_remote_code: true, max_concurrent_requests: 128, max_best_of: 2, max_stop_sequences: 4, max_top_n_tokens: 5, max_input_length: 2048, max_total_tokens: 4096, waiting_served_ratio: 1.2, max_batch_prefill_tokens: 4096, max_batch_total_tokens: None, max_waiting_tokens: 20, hostname: "0.0.0.0", port: 8880, shard_uds_path: "/tmp/text-generation-server", master_addr: "localhost", master_port: 52617, huggingface_hub_cache: Some("/local_disk0/tmp/"), weights_cache_override: None, disable_custom_kernels: false, cuda_memory_fraction: 0.9, rope_scaling: None, rope_factor: None, json_output: false, otlp_endpoint: None, cors_allow_origin: [], watermark_gamma: None, watermark_delta: None, ngrok: false, ngrok_authtoken: None, ngrok_edge: None, env: false }
2023-08-31T09:26:42.27346

In [0]:
! kill -9  $(ps aux | grep 'text-generation' | awk '{print $2}')